In [45]:
import pyscf
from pyscf import dft
from pyscf import gto
file_ = "../../../datasets/QM9/xyz/dsgdb9nsd_000001.xyz"
basis_file = "6-31g_2df_p_custom_nwchem.gbs"
xyz_data = None
with open(file_, "r") as f: 
    xyz_data = f.read()

In [46]:
# Parse the xyz_data into a format PySCF can understand
lines = xyz_data.splitlines()
num_atoms = int(lines[0])  # First line contains the number of atoms
atom_data = lines[2:2 + num_atoms]  # Skip the first two lines (header) and get atom data


atom = []
for line in atom_data:
	parts = line.split()
	atom.append(f"{parts[0]} {parts[1]} {parts[2]} {parts[3]}")  # Format: "Element x y z"
atom = "; ".join(atom)

In [47]:
import re
def parseBasis(filepath): 
    with open(filepath, "r") as f: 
        content = f.read()
    basis_set_matches = [(match.group(1), match.start()) for match in re.finditer(r"#BASIS SET:.*\n([A-Z][a-z]?)", content)]
    block_matches = [(match.start(), match.end()) for match in re.finditer(r"#BASIS SET:.*?(?=#BASIS SET:|END)", content, re.DOTALL)]
    blocks = {key:gto.basis.load(filepath, key) for key, _ in basis_set_matches}
    return blocks


In [48]:
# builtin basis
mol = pyscf.gto.M(atom=atom, basis='6-31g(2df,p)', charge=0, spin=0)

In [49]:
dict_ = parseBasis(basis_file)
mol2 = gto.M()
mol2.atom = atom
mol2.basis = dict_
mol2.build()

In [50]:
mf = dft.RKS(mol)
mf.xc = 'b3lypg'
mf.kernel()

converged SCF energy = -40.5231128730963


np.float64(-40.52311287309634)

In [51]:
mf = dft.RKS(mol2)
mf.xc = 'b3lypg'
mf.kernel()

converged SCF energy = -40.5230314176623


np.float64(-40.52303141766232)

In [52]:
from scf_guess_tools import Backend, load, calculate

backend = Backend.PY

mol1 = load(file_, backend=backend)
mol2 = load(file_, backend=backend)

In [57]:
calculate(mol1, basis_file, "minao", "dft", "b3lypg")

converged SCF energy = -40.5230314176993


In [54]:
calculate(mol1, "6-31g(2df,p)", "minao", "dft", "b3lypg")

converged SCF energy = -40.523112874433
